In [1]:
%load_ext autoreload
%autoreload 2

# TODO
1. Add datatypes
2. Create PIP package
3. Additional document in __call__()
4. READme

In [2]:
from PIL import Image, ImageDraw
from typing import Dict, Tuple, List

def draw_box(pil_img: Image, layout_info: Dict[str, Tuple[float, float, float, float]]) -> Image:
    # initialize width, height from pil_img
    width, height = pil_img.size
    draw = ImageDraw.Draw(pil_img)

    for phrase, bbox in layout_info.items():
        x_0, y_0, x_1, y_1 = bbox
        draw.rectangle([int(x_0 * width), int(y_0 * height), int(x_1 * width), int(y_1 * height)], outline='red', width=4)
        draw.text((int(x_0 * width) + 5, int(y_0 * height) + 5), phrase, fill=(255, 0, 0))
    return pil_img

def layout_index(prompt: str, layout_info: Dict[str, Tuple[float, float, float, float]]) -> Tuple[List[List[int]], Tuple[float, float, float, float]]:
    prompt_list = prompt.strip('.').split(' ')
    object_positions = []
    for obj, bbox in layout_info.items():
        obj_words = obj.split(' ')
        obj_first_index = prompt_list.index(obj_words[0]) + 1
        if len(obj_words) > 1:
            obj_last_index = prompt_list.index(obj_words[-1]) + 1
            obj_position = list(range(obj_first_index, obj_last_index + 1))
        else:
            obj_position = [obj_first_index]
        object_positions.append((obj_position, bbox))

    return object_positions

In [16]:
from layoutguidance.layout_guidance_pipeline import LayoutGuidanceStableDiffusionPipeline
from diffusers import EulerDiscreteScheduler, LMSDiscreteScheduler
import torch
seed = 33

pipe = LayoutGuidanceStableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5")
pipe = pipe.to("mps")
pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config)
# pipe.scheduler = LMSDiscreteScheduler.from_config(pipe.scheduler.config)
generator = torch.Generator(device="mps").manual_seed(seed)

ImportError: cannot import name 'draw_box' from 'layoutguidance.layout_guidance_pipeline' (/Users/njindal/build/diffusers-layout-guidance/layoutguidance/layout_guidance_pipeline.py)

In [ ]:
prompt = "A cat playing with a ball"
bboxes = [[0.55, 0.4, 0.95, 0.8]]
image = pipe(prompt, num_inference_steps=20,
             token_indices=[[2]],
             # generator=generator,
             bboxes=bboxes).images[0]
image = pipe.draw_box(image, bboxes)
display(image)

  0%|          | 0/20 [00:00<?, ?it/s]